In [3]:
import sys
import time

sys.path.append('../script')
import preprocessing as pre
import constants
import get_dataset as ds

In [4]:
# TODO: consider whether or not to split the dataset into train/test for the TensorFlow dataset generator.

feats, dialects = ds.load_windowed_dataset('mel_spectrogram', split=False, onehot=False, normalised=True)

Using normalisation method: True
Loading Ngadas: 
- Marsam.wav (time: 0.6)
- Mas_Tom_Ngadas_201102.wav (time: 0.0)
Loading Banten: 
- JWBP-20121229-A.wav (time: 0.1)
- JWBA-20100703-a.wav (time: 0.1)
- JWB-Banten_9.wav (time: 0.2)
- banten-2010-07-02-a.wav (time: 0.2)
- JWB-20090125.wav (time: 0.2)
- JWBP-20121229-B.wav (time: 0.0)
- JWB-Banten_10.wav (time: 0.0)
- JWB-20090126.wav (time: 0.2)
- JWB-20090116.wav (time: 0.1)
- JWB-Banten20100605-B.wav (time: 0.0)
- JWB-Banten_1.wav (time: 0.0)
- JWB-Banten_6.wav (time: 0.0)
- JWBA-20100703-b.wav (time: 0.1)
- JWB-Banten_4.wav (time: 0.0)
- JWB-20090117.wav (time: 0.1)
- JWB-20090118.wav (time: 0.0)
- JWB-Banten_8.wav (time: 0.0)
- JWB-20090131.wav (time: 0.3)
- JWB-20090130.wav (time: 0.1)
- JWB-Banten_11.wav (time: 0.0)
- JWB-Banten_5.wav (time: 0.0)
- JWB-20090123.wav (time: 0.0)
- JWB-20090124.wav (time: 0.1)
- JWB-20090127.wav (time: 0.2)
- JWB-Banten_19.wav (time: 0.0)
- JWB-Banten20100605-A.wav (time: 0.1)
- JWB-Banten_7.wav (time

In [5]:
feats.shape

(41594, 128, 150, 1)

In [6]:
feats.reshape(-1).shape

(798604800,)

In [7]:
dialects[0]

array(['Ngadas'], dtype='<U9')

In [8]:
feats[0,0,0]

array([5.569659e-09], dtype=float32)

In [11]:
import tensorflow as tf
from numba import jit

tf_int = lambda x: tf.train.Feature(int64_list=tf.train.Int64List(value=[x]))
tf_float32 = lambda x: tf.train.Feature(float_list=tf.train.FloatList(value=[x]))


# TensorFlow - create a TFRecords file that transform both features and dialects to a single, unified TFRecords file.
# @jit(parallel=True, fastmath=True)
# @tf.function(jit_compile=True)
def create_tf_record(features, dialects):
    # tf_features = []
    # for i in range(len(feats)): # For each features
    #     print(f'Doing {i}.', end='\r')
    #     tf_features.append(
    #         tf.train.Feature(
    #             float_list=tf.train.FloatList(
    #                 value=feats[i].reshape(-1)
    #             )
    #         )
    #     )

    # tf_features = [tf.train.Feature(float_list=tf.train.FloatList(value=x.reshape(-1))) for x in feats]

        # for j in range(len(feats[i])):  # For each 
        #     tf_feat = []
        #     for k in range(len(feats[j])):  # sample
        #         tf_sample.append(feats[i, j, k])
        #     tf_sample = tf.train.FloatList(value=tf_sample) # sample wrapping into a fixed length band
        #     tf_band.append(tf_sample)
        # tf_band = tf.train.Feature(
        #     float_list=tf_band
        # ) # band wrapping into a fixed length audio feature
        # tf_features.append(tf_band)
        
    # tf_features = tf.train.FeatureList(feature=tf_features)

    # tf_dict = {
    #     'features': tf_features,
    # }

    # tf_features = tf.train.FeatureLists(feature_list=tf_dict)
    # tf_example = tf.train.Example(features=tf_features)

    # Dialects
    # tf_dialects_binary = tf.train.BytesList(value=[dial.encode('utf-8') for dial in dialects])

    # tf_dialect = tf.train.Feature(
        # bytes_list=tf_dialects_binary
    # )

    # tf_sequence_example = tf.train.SequenceExample(
    #     context=tf.train.Features(feature={'dialect': tf_dialect}),
    #     feature_lists=tf_features
    # )

    # Output to file
    with tf.io.TFRecordWriter(constants.FEATURES_UNIFIED_DIR.__str__() + '/features-test.tfrecords') as writer:
        for i in range(len(feats)):
            print(f'Writing {i}.', end='\r')
            tf_feat = tf.train.Feature(
                float_list=tf.train.FloatList(
                    value=feats[i].reshape(-1)
                )
            )

            # print(dialects[i])
            # print(dialects[i].encode('utf-8'))
            tf_dialect = tf.train.Feature(bytes_list=tf.train.BytesList(value=[dialects[i].encode('utf-8')]))

            data = {
                'features': tf_feat,
                'dialect': tf_dialect
            }

            tf_feature = tf.train.Features(feature=data)
            example = tf.train.Example(features=tf_feature)

            writer.write(example.SerializeToString())

create_tf_record(feats, dialects.reshape(-1))

In [12]:
import gc

del feats, dialects
gc.collect()

8596